**Exemplo 02: Previsão de feedbacks de produtos B2W** 

 

Você recebeu um convite para uma consultoria, na qual deve desenvolver um modelo de previsões de feedbacks de clientes em produtos comprados na loja, que serão coletados do instagram. 

 

Os dados que você vai utilizar estão localizados em: 

https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv 

 

Na coluna 'review_title' você vai encontrar feedbacks passados dos nossos clientes em nossos produtos e, na coluna 'overall_rating', a nota que foi dada. Esse é o único dado que temos para auxiliar na criação desse modelo de previsões. 


##### IMPORTAR AS BIBLIOTECAS

In [32]:
import pandas as pd
import numpy as np
# tokeniza
from tensorflow.keras.preprocessing.text import Tokenizer #type:ignore
# Ajusta o tamanho do vetor
from tensorflow.keras.preprocessing.sequence import pad_sequences  #type:ignore
# Define o modelo de rede neural utilizada
from tensorflow.keras.models import Sequential #type:ignore
# Camadas da rede neural
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout #type:ignore
# Otimizador de taxa de aprendizado
from tensorflow.keras.optimizers import Adam #type:ignore

from sklearn.model_selection import train_test_split

##### OBTER DADOS

In [26]:
try:

    ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_title', 'overall_rating']]

    
    #print(df.columns)                
except Exception as e:
    print('ERRO AO OBTER DADOS', e)
 

KeyboardInterrupt: 

#### TRATAR DADOS

In [ ]:
try:

    # Excluindo valores nao existente (nans)
    df = df.dropna(subset=['review_title', 'overall_rating'])

    #Transformando colunas em arrays
    texts = np.array(df['review_title'])
    rating = np.array(df['overall_rating'])

    print(df['overall_rating'].unique())
    
    # Verifica se os dados estão balanceados entres as CLASSES/CATEGORIAS
    # Nesse exmplo a classe 2 tem bem menos dados. 
    print(df['overall_rating'].value_counts())

                     
except Exception as e:
    print('ERRO AO OBTER DADOS', e)
   

[4 5 1 2 3]
overall_rating
5    47856
4    32285
1    27280
3    16279
2     8371
Name: count, dtype: int64


#### VETORIZAR

In [ ]:
try:

    # Há bibliotecas de sinonimo para aprimorar o modelo
    # Hugging face dicionário de contextos amplos - gratuito
    
    # Passo 1: tokenizar
    tokenizer = Tokenizer()

    # Passo 2: Criar o dicionário
    # fit_on_texts: Cria o vacabulário, através do dicionário
    # associando cada token a um indice numérico
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo indice numerico
    tokenizer.fit_on_texts(texts)

    #print(tokenizer)
    # Passo3: Vetorizar, ou seja, transformar os tokens em números a partir do dicionario criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)
    #print(vetores)
    # Passo 4: Padronização do tamanho do vetor - pad
    padded_vetores = pad_sequences(vetores) 

    print(padded_vetores)
    
                     
except Exception as e:
    print('ERRO AO VETORIZAR TEXTOS', e)
    exit()

[[   0    0    0 ...    0    0    3]
 [   0    0    0 ... 2620   30   16]
 [   0    0    0 ...  349   45  155]
 ...
 [   0    0    0 ...    0    9    1]
 [   0    0    0 ...    4   19    3]
 [   0    0    0 ...    1    4   51]]


##### CONSTRUIR A REDE NEURAL

In [33]:
try:

    # Constantes do modelo

    # 1ª Constante: Tamanho do vacabulário (tamanho do dicionario do modelo de contexto)
    VOCAB_SIZE = len(tokenizer.word_index) + 1

    # 2ª Constante: Tamanho máximo da sequencia
    # É o comprimento máximo de um texto
    MAX_SEQUENCE_LENGTH = padded_vetores.shape[1] # linha 0 coluna 1 a quantidade de colunas vai mostrar o comprimento maximo do vetor (a qtde de palavras)

    # 3ª Constante: Tamanho do vetor de entrada
    # A literatura recomenda que seja iniciado pela quantidade igual a raiz quadrada do tamanho do vocabulário
    # Se o volume de dados for de larga escala, pode-se testar iniciando com um tamanho maior
    # Se o volume de dados for muito pequeno, pode-se testar iniciando com um tamanho menor
    # Cuidado com o overfitting, que é quando o modelo aprende demais e começa a perder a capacidade de generalizar com novos dados
    # Não consegue observar todas as diferenças textuais
    # Overfitting pode ser observado no treino da rede neural
    VETOR_LENGTH = int(np.sqrt(VOCAB_SIZE))

    # Inicia-se a construção da rede neural
    # Sequential é fluxo linear de camadas (conforme visto na Aula02_RNA.pptx)
    # São processadas em ordem
    model = Sequential()

    # Camada de entrada
    # Embedding, na qual os vetores de texto são inseridos
    model.add(Embedding(input_dim=VOCAB_SIZE,
                        output_dim =VETOR_LENGTH, # Output da camada de entrada. input pra camada oculta
                        input_length=MAX_SEQUENCE_LENGTH))
    
    # Camada oculta ou intermediária
    # LSTM - long short term memory, em portugues " memoria de curto e longo prazo"
    # É onde a magia acontece. É onde o modelo treina baseado nos seus vetores
    # Números de unidades de memória, que é a quantidade de neuronios. Quanto mais neuronios, maior a acurácia
    # No primeiro TESTE experimente somente com uma camada! Cuidado com o overfitting!
    # Se for necessário adicionar mais camadas, basta repetir o comando abaixo
    
    # Primeira camada oculta
    model.add(LSTM(128)) # Uma camada com 128 neuronios

    # Se necessário adicionar outra camada oculta, repita model.add(LSTM(qtde de neuronios))

    # Camada de saída - Camada densa
    # Na classificação precisa ajustar para a quantidade de CLASSES/categorias de saída
    # Em RNA DE CLASSIFICAO (CLASSES/CATEGORIA)
    # a função de ativação mais utilizda é a softmax
    # fç de ativação é um calculo matematico
    #que vai determinar a saida de cada neuronio
    # softmax é aum fç que transforma os valores de saida em PROBABILIDADES que vao de 0 a 1
    # a soma das classes, no nosso caso 5 classes, será igual 1
    # Aplicar função de ativação para que o resultado das classses de saída somados deem 1 e fique mais facil de ver a saida
    # A outra fç mais utilizada é a RELU QUE RETORNA 0 para saidas negativa e o valor original para as saidas maiores do que 0
    # o que faz ela ser, mais indicada para modelos de regressao
    model.add(Dense(5, activation='softmax'))

    # Dropout é uma tecnicade regularização do resultado, pra minimizar o overfitting
    #ele desabilita neuronios aleatoriamente, justamente para tentar minimizar o overfitting
    # cuidado com o underfitting, pq reduzir demais o aprendizado para reduzir o verfitting pode levar a aprender menos d que deveria
    model.add(Dropout(0.05)) # desativar 5% dos neuronios aleatoriamente é uma função degenerativa como uma doenca degenerativa, perde neuronios

    # Construir o modelo
    # É literalmente pegar as definições anteriores e construir o modelo
    # input_shape: é o formato dos dados de entrada e ainda o tamanho máximo do texto (MAX_SEQUENCE_LENGHT)

    model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))

    # Otimizador de taxa de aprendizado. 
    # Importante para ajustar em casos de overfitting
    # Adam é o otimizador que ajusta essa taxa de aprendizado
    # parametro learn_rating: Quanto menor, melhor o aprendizado. Menos risco de overfitting 
    otimizador = Adam(learning_rate=0.0001)

    # Compilar o modelo
    # Verificar se há ou não algum erro
    # É informado o otimizador e a métrica de perda (LOSS)
    # loss - erro quadro médio (mean_squared_error)
    model.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')

    model.summary()
    print('Modelo configurado e criado')


except Exception as e:
    print('ERRO AO CONSTRUIR A REDE NEURAL', e)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 31, 124)        │     1,936,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 128)            │       129,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 5)              │           645 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,067,061 (7.89 MB)

 Trainable params: 2,067,061 (7.89 MB)

 Non-trainable params: 0 (0.00 B)

Modelo configurado e criado


##### TREINAR O MODELO

In [34]:
try:

    X_train, X_test, y_train, y_test = train_test_split(
        padded_vetores,
        rating,
        test_size=0.2,
        random_state=42
    )

    # Qdo a gente constroi um RNA de classificação
    # Se o meu dado tiver 0 na classe, eu não preciso ajustar, mas se começar em 1, precisa desse ajuste
    # Os rótulos que a RNA atribui [0,1,2,3,4] e os meus dados [1,2,3,4,5] 
    y_train_adjusted = y_train - 1
    y_test_adjusted = y_test - 1

    # Aplicar pesos às categorias
    # Verifica o tamanho do dataframe e divide pelo tamanho das categorias.
    # quanto menor a quantidade de dados, maiores os pesos, justamente para eles conseguirem ser mais expressivos
    pesos = len(df['overall_rating'])/df['overall_rating'].value_counts()
    #print(pesos)

    # O treino da rede neural
    model.fit(
        X_train,
        y_train_adjusted,
        epochs=5,
        batch_size=32, # Quanto maior o tamanho do batch, menor o aprendizado, porém evita overfitting
        # Quanto menor o tamnho da batch o modelo tende a um maior aprendizado (logica: ler um livro mais lento, vc compreende melhor)
        class_weight = pesos.to_dict(),
        validation_data=(X_test, y_test_adjusted))
    
    # a loss precisa ficar sempre reduzindo, se em algumm momento alumentar, tem um sinal de overfitting
    # Na regressão precisa comparar a loss com a val_loss
    
except Exception as e:
    print('ERRO AO TREINAR O MODELO', e)

Epoch 1/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 52s 15ms/step - loss: 11.3316 - val_loss: 1.2235
Epoch 2/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 50s 15ms/step - loss: 9.9711 - val_loss: 1.2000
Epoch 3/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - loss: 9.9085 - val_loss: 1.0703
Epoch 4/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 53s 16ms/step - loss: 9.5566 - val_loss: 1.1210
Epoch 5/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - loss: 9.4984 - val_loss: 1.1004


##### TESTAR O MODELO

In [36]:
try:

    novos_textos = [
        'Muito bom, gostei bastante. Top demais! Compensa muito!',
         "Muito bom, Americanas. Só faz besteira. Não recomento",
    ]

    novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
    novas_sequencias_padded = pad_sequences(novas_sequencias)

    predicoes = model.predict(novas_sequencias_padded)

    # Formatar valores de saida
    np.set_printoptions(suppress=True, precision=4)

    print("Previsões:", predicoes) # precisa somar um para voltar as classes normais predicoes + 1 se nao usar a função de ativação
    
except Exception as e:
    print('ERRO AO TESTAR O MODELO', e)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Previsões: [[0.0001 0.0004 0.0085 0.2633 0.7277]
 [0.3491 0.2292 0.1714 0.1456 0.1048]]
